In [ ]:
label_classes = {
    "圆形": 0,
    "未熔合": 1,
    "未焊透": 2,
    "条形": 3,
    "非焊缝区域": 4,
    "内凹": 5,
    "咬边": 6,
    "合格": 7,
}

In [ ]:
from PIL import Image
import numpy as np
from pathlib import Path

# Load image
def extract_coords(img_path: Path):
    bounding_boxes = []
    image = Image.open(img_path).convert('RGB')
    pixels = np.array(image)

    # Define red color threshold
    red_threshold = (pixels[:, :, 0] > 150) & (pixels[:, :, 1] < 100) & (pixels[:, :, 2] < 100)

    # Get coordinates of red pixels
    red_coords = np.column_stack(np.where(red_threshold))

    # If there are red pixels, find bounding box
    if red_coords.size > 0:
        min_y, min_x = red_coords.min(axis=0)
        max_y, max_x = red_coords.max(axis=0)
        bounding_boxes.append((int(min_x), int(min_y), int(max_x), int(max_y)))
    return bounding_boxes
 
from PIL import ImageDraw
def check_result_correct(img_path: Path, bounding_boxes):
    img = Image.open(img_path)
    img_draw = ImageDraw.Draw(img)
    for min_x, min_y, max_x, max_y in bounding_boxes:
        img_draw.rectangle([min_x, min_y, max_x, max_y], outline='red', width=2)
    return img
    
coords_folder = Path("bounding_boxes")
coords_folder.mkdir(exist_ok=True)
marked_folder = Path("jpg_marked")
origin_folder = Path("jpg_data")
tgt_folder = Path("jpg_bounding_boxes")
tgt_folder.mkdir(exist_ok=True)

In [ ]:
files_to_recheck=[]
for marked_file in marked_folder.rglob("**/*.jpg"):
    rel_path = marked_file.relative_to(marked_folder)
    origin_file = origin_folder / rel_path
    if origin_file.exists():
        marked_img = Image.open(marked_file)
        marked_width = marked_img.width
        origin_width = Image.open(origin_file).width
        if marked_width != origin_width:
            files_to_recheck.append(marked_file.name)
            marked_height = marked_img.height
            if marked_width > marked_height:
                marked_img = marked_img.transpose(Image.Transpose.ROTATE_270)
            else:
                marked_img = marked_img.transpose(Image.Transpose.ROTATE_90)
            marked_img.save(marked_file)

In [ ]:
files_to_recheck

In [ ]:
from PIL import Image
import numpy as np
from pathlib import Path

def generate_scaled_masks(img_path: Path, bounding_box):
    image = Image.open(img_path).convert('RGB')
    pixels = np.array(image)
    img_height, img_width = pixels.shape[:2]
    
    min_x, min_y, max_x, max_y = bounding_box
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2
    orig_width = max_x - min_x
    orig_height = max_y - min_y

    scale = 1
    masked_images = []

    while True:
        new_width = orig_width * scale
        new_height = orig_height * scale

        new_min_x = center_x - new_width // 2
        new_max_x = center_x + new_width // 2
        new_min_y = center_y - new_height // 2
        new_max_y = center_y + new_height // 2

        # Check if the scaled bounding box is within image boundaries
        if new_min_x < 0 or new_min_y < 0 or new_max_x > img_width or new_max_y > img_height:
            break

        # Create mask
        mask = np.zeros_like(pixels)
        mask[int(new_min_y):int(new_max_y), int(new_min_x):int(new_max_x)] = pixels[int(new_min_y):int(new_max_y), int(new_min_x):int(new_max_x)]
        masked_image = Image.fromarray(mask)
        masked_images.append((scale, masked_image))

        scale += 1

    return masked_images
 

In [ ]:
from PIL import Image
import numpy as np
from pathlib import Path

def generate_scaled_masks_with_noise(img_path: Path, bounding_box):
    image = Image.open(img_path).convert('RGB')
    pixels = np.array(image)
    img_height, img_width = pixels.shape[:2]

    min_x, min_y, max_x, max_y = bounding_box
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2
    orig_width = max_x - min_x
    orig_height = max_y - min_y

    scale = 1
    masked_images = []

    while True:
        new_width = orig_width * scale
        new_height = orig_height * scale

        new_min_x = center_x - new_width // 2
        new_max_x = center_x + new_width // 2
        new_min_y = center_y - new_height // 2
        new_max_y = center_y + new_height // 2

        # Check if the scaled bounding box is within image boundaries
        if new_min_x < 0 or new_min_y < 0 or new_max_x > img_width or new_max_y > img_height:
            break

        # Create a copy of the original image
        noisy_image = np.copy(pixels)

        # Generate random noise
        noise = np.random.randint(0, 256, size=pixels.shape, dtype=np.uint8)

        # Apply noise outside the scaled bounding box
        mask = np.ones((img_height, img_width), dtype=bool)
        mask[int(new_min_y):int(new_max_y), int(new_min_x):int(new_max_x)] = False
        noisy_image[mask] = noise[mask]

        masked_image = Image.fromarray(noisy_image)
        masked_images.append((scale, masked_image))

        scale += 1

    return masked_images
 

In [ ]:
for img_file in marked_folder.rglob("**/*.jpg"):
    rel_path = img_file.relative_to(marked_folder)
    img_file_to_slice = origin_folder / rel_path
    coords = extract_coords(img_file)
    for index, (min_x, min_y, max_x, max_y) in enumerate(coords):
        if not img_file_to_slice.exists():
            continue
        masked_images = generate_scaled_masks_with_noise(img_file_to_slice, (min_x, min_y, max_x, max_y))
        for scale, img in masked_images:
            tgt_file = tgt_folder / f"{img_file.stem}_{index}{scale}.jpg"
            img.save(tgt_file)

        coords_txt = tgt_folder / f"{img_file.stem}_{index}.txt"
        with open(coords_txt, "w") as f:
            line = f"{min_x} {min_y} {max_x} {max_y}"
            f.write(line)
    # img_file.unlink()

In [ ]:
def find_tag(file:Path):
    for k, v in label_classes.items():
        if k in str(file.absolute()):
            return v

In [ ]:
from PIL import Image

def transform_to_yolo_format(src_img: Image, class_id: int, min_x: int, min_y: int, max_x: int, max_y: int):
    # Get image dimensions
    img_height = src_img.height
    img_width = src_img.width

    # Calculate the center of the bounding box
    center_x = (min_x + max_x) // 2
    center_y = (min_y + max_y) // 2
    width = max_x - min_x
    height = max_y - min_y

    # Normalize the values to [0, 1] range
    normalized_center_x = center_x / img_width
    normalized_center_y = center_y / img_height
    normalized_width = width / img_width
    normalized_height = height / img_height

    assert normalized_center_x <= 1

    # Prepare the YOLO format string
    yolo_format_string = f"{class_id} {normalized_center_x} {normalized_center_y} {normalized_width} {normalized_height}"

    return yolo_format_string

In [ ]:
# import shutil

# for img_file in marked_folder.rglob("**/*.jpg"):
#     rel_path = img_file.relative_to(marked_folder)
#     img_file_to_slice = origin_folder / rel_path
#     coords = extract_coords(img_file)
#     coords_txt = tgt_folder / f"{img_file.stem}.txt"
#     coords_img = tgt_folder / f"{img_file.stem}.jpg"
#     lines = []
#     print(img_file)
#     for index, (min_x, min_y, max_x, max_y) in enumerate(coords):
#         img = Image.open(img_file)
#         tag = find_tag(img_file)
#         if tag is None:
#             continue
#         if img_file_to_slice.exists():
#             line = transform_to_yolo_format(
#                 img,
#                 tag,
#                 min_x,
#                 min_y,
#                 max_x,
#                 max_y,
#             )
#             lines.append(line)
#             with open(coords_txt, "w") as f:
#                 f.write("\n".join(lines))
#             shutil.copyfile(img_file_to_slice, coords_img)

In [ ]:
# for img_file in origin_folder.rglob("**/*.jpg"):
#     if "合格" in str(img_file.absolute()):
#         coords_txt = tgt_folder / f"{img_file.stem}.txt"
#         coords_img = tgt_folder / f"{img_file.stem}.jpg"
#         lines = []
#         with open(coords_txt, "w") as f:
#             f.write("\n".join(lines))
#         shutil.copyfile(img_file_to_slice, coords_img)

In [ ]:
import shutil
train_folder = Path("official_images/train_origin")
for file in tgt_folder.rglob("**/*.jpg"):
    file_prefix = file.name.split("_")[0]
    for train_file in train_folder.rglob("**/*.jpg"):
        if train_file.stem in file_prefix:
            tgt_path = train_file.parent / file.name
            print(tgt_path)
            shutil.copyfile(file, tgt_path)


In [ ]:
from pydicom import dcmread
from PIL import Image, ImageOps
import numpy as np

def center_pad_to_square(pil_img: Image, fill_color=0) -> Image.Image:

    # Get original dimensions
    origin_width, origin_height = pil_img.size
    if origin_width < origin_height:
        pil_img = pil_img.transpose(Image.Transpose.ROTATE_90)
    width, height = pil_img.size
    max_side = width

    # Calculate padding
    pad_left = (max_side - width) // 2
    pad_right = max_side - width - pad_left
    pad_top = (max_side - height) // 2
    pad_bottom = max_side - height - pad_top

    # Apply center padding
    padded_img = ImageOps.expand(pil_img, border=(pad_left, pad_top, pad_right, pad_bottom), fill=fill_color)
    return padded_img
 

In [ ]:
for file in train_folder.rglob("**/*.jpg"):
    img = center_pad_to_square(Image.open(file))
    img = img.resize((512, 512))
    img.save(file)

In [ ]:
valid_folder = Path("official_images/valid")
for file in valid_folder.rglob("**/*.jpg"):
    img = center_pad_to_square(Image.open(file))
    img = img.resize((512, 512))
    img.save(file)